 **IMPORT LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
import gradio as gr
import requests
from geopy.geocoders import Nominatim
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


 **LOAD DATASET**

In [2]:
df = pd.read_csv("/content/project_dataset_final.csv")

In [3]:
df.head()

,Soil_Type,Nitrogen,Phosphorus,Potassium,Soil_pH,Temperature_C,Rainfall_mm,Water_Availability,Season,Crop_Duration_Days,Crop_Type,Recommended_Variety,Yield_Class,Yield_Profit_Range_%
0,Medium to Deep Black,22.2,66.6,41.7,7.45,25.9,742.2,Medium,Kharif,99,Soybean,JS 335,Medium,25-35
1,Medium to Deep Black,20.5,75.8,44.2,7.50,26.3,747.4,Medium,Kharif,98,Soybean,JS 335,Medium,25-35
2,Medium to Deep Black,25.3,64.0,54.6,6.67,34.5,626.9,Medium,Kharif,98,Soybean,JS 335,Medium,25-35
3,Medium to Deep Black,20.5,64.9,42.2,6.96,30.4,827.7,Medium,Kharif,98,Soybean,JS 335,Medium,25-35
4,Medium to Deep Black,27.8,64.8,54.2,7.24,30.7,856.5,Medium,Kharif,96,Soybean,JS 335,Medium,25-35


In [4]:
df.describe()

,Nitrogen,Phosphorus,Potassium,Soil_pH,Temperature_C,Rainfall_mm,Crop_Duration_Days
count,1150.000000,1150.000000,1150.000000,1150.000000,1150.000000,1150.000000,1150.000000
mean,80.045661,58.751609,69.952252,6.775243,25.982696,581.607565,148.913913
std,58.519143,14.847606,27.731706,0.492529,4.981619,166.132871,72.781170
min,15.000000,30.000000,30.700000,5.500000,14.100000,250.100000,70.000000
25%,22.800000,47.700000,50.377500,6.420000,22.500000,434.525000,100.000000
50%,93.800000,56.600000,59.500000,6.790000,26.400000,595.600000,116.000000
75%,134.300000,69.100000,84.400000,7.170000,30.000000,709.950000,179.000000
max,179.900000,99.800000,149.800000,7.970000,36.000000,896.100000,360.000000


**ENCODING**

In [5]:
encoders = {}
cat_cols = ["Soil_Type","Water_Availability","Crop_Type","Recommended_Variety","Yield_Class"]
df = pd.read_csv("/content/project_dataset_final.csv")

for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).apply(lambda x: x.split('(')[0].strip() if '(' in x else x)
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        encoders[col] = le

**CROP → VARIETY MAPPING**

In [6]:
crop_variety_map = {}

for _, row in df.iterrows():
    crop = encoders["Crop_Type"].inverse_transform([row["Crop_Type"]])[0]
    variety = encoders["Recommended_Variety"].inverse_transform([row["Recommended_Variety"]])[0]

    if crop not in crop_variety_map:
        crop_variety_map[crop] = set()

    crop_variety_map[crop].add(variety)

for crop in crop_variety_map:
    crop_variety_map[crop] = list(crop_variety_map[crop])


**ML DATA & PREPROCESSING**

In [7]:
X = df[["Soil_Type","Nitrogen","Phosphorus","Potassium","Soil_pH",
        "Temperature_C","Rainfall_mm","Water_Availability","Crop_Type"]]

y_variety = df["Recommended_Variety"]
y_yield = df["Yield_Class"]
y_profit = df["Yield_Profit_Range_%"]
y_days = df["Crop_Duration_Days"]


def parse_profit_range(profit_range_str):
    if isinstance(profit_range_str, str) and '-' in profit_range_str:
        try:
            min_val, max_val = map(int, profit_range_str.split('-'))
            return (min_val + max_val) / 2
        except ValueError:
            return None
    return profit_range_str

y_profit = y_profit.apply(parse_profit_range)
y_profit = pd.to_numeric(y_profit, errors='coerce')
cleaned_indices = y_profit.dropna().index
y_profit = y_profit.dropna()
X = X.loc[cleaned_indices]
y_variety = y_variety.loc[cleaned_indices]
y_yield = y_yield.loc[cleaned_indices]
y_days = y_days.loc[cleaned_indices]


**MODEL** **TRAINING**

In [8]:
model_variety = RandomForestClassifier()
model_yield = RandomForestClassifier()
model_profit = RandomForestRegressor()
model_days = RandomForestRegressor()

model_variety.fit(X, y_variety)
model_yield.fit(X, y_yield)
model_profit.fit(X, y_profit)
model_days.fit(X, y_days)


RandomForestRegressor()

**DATA SPLITTING AND MODEL EVALUATION**

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_variety_train, y_variety_test, \
y_yield_train, y_yield_test, y_profit_train, y_profit_test, \
y_days_train, y_days_test = train_test_split(
    X, y_variety, y_yield, y_profit, y_days,
    test_size=0.2, random_state=42
)

print("Data split into training and testing sets successfully.")
print(f"Training set size: {len(X_train)} samples")
print(f"Testing set size: {len(X_test)} samples")

Data split into training and testing sets successfully.
Training set size: 648 samples
Testing set size: 162 samples


**RETRAINING MODELS ON SPLIT DATA**

In [10]:
# Retrain models on the training data
model_variety_split = RandomForestClassifier(random_state=42)
model_yield_split = RandomForestClassifier(random_state=42)
model_profit_split = RandomForestRegressor(random_state=42)
model_days_split = RandomForestRegressor(random_state=42)

model_variety_split.fit(X_train, y_variety_train)
model_yield_split.fit(X_train, y_yield_train)
model_profit_split.fit(X_train, y_profit_train)
model_days_split.fit(X_train, y_days_train)

print("Models retrained on training data successfully.")

Models retrained on training data successfully.


**MODEL EVALUATION**

In [11]:
# Make predictions on the test data
y_variety_pred = model_variety_split.predict(X_test)
y_yield_pred = model_yield_split.predict(X_test)
y_profit_pred = model_profit_split.predict(X_test)
y_days_pred = model_days_split.predict(X_test)

# Evaluate classification models using accuracy_score
accuracy_variety = accuracy_score(y_variety_test, y_variety_pred)
accuracy_yield = accuracy_score(y_yield_test, y_yield_pred)

print(f"Accuracy Score for Recommended Variety: {accuracy_variety:.4f}")
print(f"Accuracy Score for Yield Class: {accuracy_yield:.4f}")


Accuracy Score for Recommended Variety: 0.8025
Accuracy Score for Yield Class: 0.8580


### Random Forest Evaluation

In [12]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

print("--- Random Forest Model Evaluation ---")

# Re-split data to ensure consistency with previous evaluations (if not already done)
# Assuming X, y_variety, y_yield, y_profit, y_days are still in scope and correctly processed
X_train, X_test, y_variety_train, y_variety_test, \
y_yield_train, y_yield_test, y_profit_train, y_profit_test, \
y_days_train, y_days_test = train_test_split(
    X, y_variety, y_yield, y_profit, y_days,
    test_size=0.2, random_state=42
)

# --- Random Forest Classifier for Variety ---
model_variety_rf = RandomForestClassifier(random_state=42)
model_variety_rf.fit(X_train, y_variety_train)
y_variety_pred_rf = model_variety_rf.predict(X_test)
accuracy_variety_rf = accuracy_score(y_variety_test, y_variety_pred_rf)
print(f"Random Forest Accuracy for Recommended Variety: {accuracy_variety_rf:.4f}")

# --- Random Forest Classifier for Yield Class ---
model_yield_rf = RandomForestClassifier(random_state=42)
model_yield_rf.fit(X_train, y_yield_train)
y_yield_pred_rf = model_yield_rf.predict(X_test)
accuracy_yield_rf = accuracy_score(y_yield_test, y_yield_pred_rf)
print(f"Random Forest Accuracy for Yield Class: {accuracy_yield_rf:.4f}")

# --- Random Forest Regressor for Profit ---
model_profit_rf = RandomForestRegressor(random_state=42)
model_profit_rf.fit(X_train, y_profit_train)
y_profit_pred_rf = model_profit_rf.predict(X_test)
r2_profit_rf = r2_score(y_profit_test, y_profit_pred_rf)
mse_profit_rf = mean_squared_error(y_profit_test, y_profit_pred_rf)
print(f"Random Forest R-squared for Yield Profit: {r2_profit_rf:.4f}")
print(f"Random Forest Mean Squared Error for Yield Profit: {mse_profit_rf:.4f}")

# --- Random Forest Regressor for Crop Duration Days ---
model_days_rf = RandomForestRegressor(random_state=42)
model_days_rf.fit(X_train, y_days_train)
y_days_pred_rf = model_days_rf.predict(X_test)
r2_days_rf = r2_score(y_days_test, y_days_pred_rf)
mse_days_rf = mean_squared_error(y_days_test, y_days_pred_rf)
print(f"Random Forest R-squared for Crop Duration Days: {r2_days_rf:.4f}")


--- Random Forest Model Evaluation ---
Random Forest Accuracy for Recommended Variety: 0.8025
Random Forest Accuracy for Yield Class: 0.8580
Random Forest R-squared for Yield Profit: 0.8920
Random Forest Mean Squared Error for Yield Profit: 13.2009
Random Forest R-squared for Crop Duration Days: 0.9784


**LIVE** **WEATHER**

In [13]:
def get_lat_lon(city):
    geolocator = Nominatim(user_agent="crop_ai")
    location = geolocator.geocode(city, timeout=10)
    if location is None:
        raise Exception("City not found")
    return location.latitude, location.longitude

def get_weather(city):
    try:
        lat, lon = get_lat_lon(city)
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,precipitation"
        response = requests.get(url, timeout=10).json()
        if "current" not in response:
            raise Exception(f"Weather API response error: {response.get('reason', 'Unknown error')}")
        current = response["current"]
        return current["temperature_2m"], current["precipitation"], current["relative_humidity_2m"]
    except Exception as e:
        print(f"Error fetching weather for {city}: {e}")
        return None, None, None



**Define Crop-Soil Suitability Map**

In [14]:
# This map defines general suitability. You can expand this based on agricultural expert knowledge.
crop_soil_suitability_map = {
    "Soybean": ["Medium to Deep Black", "Deep Black","Black cotton","Medium Black"],
    "Wheat": ["Clay Loam", "Loam", "Sandy Loam", "Well Drained Loam"],
    "Corn(Maize)": ["Sandy Loam", "Loam", "Medium Black Soil"],
    "Tomato": ["Sandy Loam", "Loam"],
    "Potato": ["Sandy Loam", "Loam"],
    "Onion": ["Sandy Loam to Loam", "Loam"],
    "Pomegranate": ["Deep Loamy","Well Drained Loam","Deep Well Drained","Medium Well Drained","Sandy Loam"],
    "Dragon Fruit": ["Well-drained sandy loam", "Loam", "sandy loam"],
    "Watermelon": ["Light Black Soil", "Loamy", "Sandy Loam"]
}

print("Crop-Soil Suitability Map defined.")

Crop-Soil Suitability Map defined.


**AI** **PREDICTION** **FUNCTION**

In [15]:
def crop_ai(district, soil, n, p, k, ph, water, crop):

    try:
        temp, rain, hum = get_weather(district)
        if '(' in soil:
            soil_display = soil.split('(')[0].strip()
            soil = soil.split('(')[0].strip()
        else:
            soil_display = soil

        if '(' in water:
            water = water.split('(')[0].strip()
        if '(' in crop:
            crop_display = crop.split('(')[0].strip()
            crop = crop.split('(')[0].strip()
        else:
            crop_display = crop

        # Validate and encode Soil_Type
        if soil not in encoders["Soil_Type"].classes_:
            raise ValueError(f"Soil type '{soil}' is not a recognized option. Please select a valid soil type from the dropdown.")
        soil_encoded = encoders["Soil_Type"].transform([soil])[0]

        # Validate and encode Water_Availability
        if water not in encoders["Water_Availability"].classes_:
            raise ValueError(f"Water availability '{water}' is not a recognized option. Please select a valid option from the dropdown.")
        water_encoded = encoders["Water_Availability"].transform([water])[0]

        # Validate and encode Crop_Type
        if crop not in encoders["Crop_Type"].classes_:
            raise ValueError(f"Crop type '{crop}' is not a recognized option. Please select a valid crop type from the dropdown.")
        crop_encoded = encoders["Crop_Type"].transform([crop])[0]

        # --- New Suitability Check ---
        if crop_display in crop_soil_suitability_map:
            if soil_display not in crop_soil_suitability_map[crop_display]:
                raise ValueError(f"The selected Soil Type '{soil_display}' is generally NOT suitable for Crop Type '{crop_display}'. Please choose a different combination or consult local agricultural experts.")
        # --- End New Suitability Check ---

        data = [[soil_encoded, n, p, k, ph, temp, rain, water_encoded, crop_encoded]]

        # Variety prediction
        all_probs = model_variety.predict_proba(data)[0]
        model_class_to_prob = {cls: prob for cls, prob in zip(model_variety.classes_, all_probs)}
        allowed_varieties_names = crop_variety_map[crop]
        allowed_varieties_encoded = encoders["Recommended_Variety"].transform(allowed_varieties_names)

        filtered_variety_probs = {}
        for encoded_val in allowed_varieties_encoded:
            if encoded_val in model_class_to_prob:
                filtered_variety_probs[encoded_val] = model_class_to_prob[encoded_val]

        if not filtered_variety_probs:
            if allowed_varieties_encoded.size > 0:
                v_pred = allowed_varieties_encoded[0]
            else:
                raise Exception(f"No valid varieties found for crop: {crop}. Please check dataset consistency.")
        else:
            v_pred = max(filtered_variety_probs, key=filtered_variety_probs.get)

        # Other predictions
        y_pred = model_yield.predict(data)[0]
        p_pred = model_profit.predict(data)[0]
        d_pred = model_days.predict(data)[0]

        variety_name = encoders["Recommended_Variety"].inverse_transform([v_pred])[0]
        yield_class_name = encoders["Yield_Class"].inverse_transform([y_pred])[0]

        return f"""
## ✨ AI Recommendation

️ District: {district}
☁️ Temperature: {temp} °C
☔️ Rainfall: {rain} mm
️ Humidity: {hum} %

 Recommended Variety: {variety_name}
️ Yield Class: {yield_class_name}
️ Profit Range: {round(p_pred,2)} %
️ Crop Duration: {int(d_pred)} days
"""
    except ValueError as ve:
        return f"❌ Input Error: {str(ve)}"
    except Exception as e:
        return f"❌ An unexpected error occurred: {str(e)}. Please check your inputs or try again later."

**GOOGLE** **FORM** **INTEGRATION**

In [16]:

FORM_URL = "https://docs.google.com/spreadsheets/d/120fGf13yaZGtlwtHc1tIl0ReByrG37tB0uL_HpKTfOs/export?format=csv&gid=246251299"

def fetch_and_predict_latest_submission():
    try:
        google_df = pd.read_csv(FORM_URL)
        if google_df.empty:
            return "❌ No submissions found in the Google Form."

        row = google_df.iloc[-1]

        district = row["District (Maharashtra)\n"].strip()
        soil = row["Soil Type\n"].strip()
        n = float(row["Nitrogen (kg/ha)\n"])
        p = float(row["Phosphorus (kg/ha)"])
        k = float(row["Potassium (kg/ha)"])
        ph = float(row["Soil PH"])
        water = row["Water Availability"].strip()
        crop = row[" Crop Type "].strip()
        return crop_ai(district, soil, n, p, k, ph, water, crop)

    except Exception as e:
        return f"❌ Error fetching or processing Google Sheet data: {str(e)}"


**GRADIO** **UI**

In [17]:
soil_types_choices = list(encoders["Soil_Type"].classes_)
water_options_choices = list(encoders["Water_Availability"].classes_)
crop_types_choices = list(encoders["Crop_Type"].classes_)

custom_css = """
    .gr-box {border-radius: 12px; padding: 12px;}
    .gr-button {background-color: #4CAF50; color: white; font-weight: bold;}
    .gr-button:hover {background-color: #45a049;}
    .output-box {background-color: #f0f8ff; border: 2px solid #4CAF50; border-radius: 12px; padding: 12px; font-weight:bold;}
    .section-header {color: #2E8B57; font-weight: bold;}
    .gr-row {gap: 20px;}

"""

with gr.Blocks(css=custom_css) as iface:

    gr.Markdown("##  Smart Crop Variety Recommendation Dashboard", elem_classes="section-header")

    with gr.Tab("Manual Entry📝"):
        with gr.Row():
            with gr.Column():
                district = gr.Textbox(label="village", placeholder="Enter village name")
                soil = gr.Dropdown(choices=soil_types_choices, label="Soil Type", value="Medium to Deep Black")
                n = gr.Number(label="Nitrogen (kg/ha)", value=22)
                p = gr.Number(label="Phosphorus (kg/ha)", value=66)
            with gr.Column():
                k = gr.Number(label="Potassium (kg/ha)", value=41)
                ph = gr.Number(label="Soil pH", value=7.0)
                water = gr.Dropdown(choices=water_options_choices, label="Water Availability", value="Medium")
                crop = gr.Dropdown(choices=crop_types_choices, label="Crop Type", value="Soybean")

        manual_btn = gr.Button(" Predict Variety")
        manual_output = gr.Markdown(elem_classes="output-box")
        manual_btn.click(crop_ai, inputs=[district, soil, n, p, k, ph, water, crop], outputs=manual_output)

    with gr.Tab("Latest Google Form Submission 📋"):
        gr.Markdown("Click the button below to fetch the latest submission from the linked Google Form and get a prediction.")
        auto_btn = gr.Button(" Fetch & Predict Latest Submission")
        auto_output = gr.Markdown(elem_classes="output-box")
        auto_btn.click(fetch_and_predict_latest_submission, inputs=[], outputs=auto_output)

iface.launch()

/tmp/ipython-input-2132232618.py:15: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css) as iface:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff6435357b52fe6ef0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**KrushiAI – Climate Risk Advisory**

In [18]:
import gradio as gr

climate_rules = {
    "heavy_rain": {
        "check": lambda r, t, h: r > 150,
        "risk": "HIGH",
        "advice_en": "Ensure proper drainage and avoid fertilizer application.",
        "issue_title_mr": "अतिवृष्टी",
        "advice_mr": "योग्य निचरा सुनिश्चित करा आणि खतांचा वापर टाळा."
    },
    "low_rainfall": {
        "check": lambda r, t, h: r < 50,
        "risk": "HIGH",
        "advice_en": "Use drip irrigation and mulching.",
        "issue_title_mr": "कमी पाऊस",
        "advice_mr": "ठिबक सिंचन आणि मल्चिंगचा वापर करा."
    },
    "heat_wave": {
        "check": lambda r, t, h: t > 38,
        "risk": "MEDIUM",
        "advice_en": "Irrigate during evening and apply mulching.",
        "issue_title_mr": "उष्णतेची लाट",
        "advice_mr": "संध्याकाळी सिंचन करा आणि मल्चिंगचा वापर करा."
    },
    "cold_stress": {
        "check": lambda r, t, h: t < 15,
        "risk": "MEDIUM",
        "advice_en": "Cover crops and delay sowing.",
        "issue_title_mr": "थंडीचा ताण",
        "advice_mr": "पिकांना आच्छादन द्या आणि पेरणीला उशीर करा."
    },
    "high_humidity": {
        "check": lambda r, t, h: h > 80,
        "risk": "MEDIUM",
        "advice_en": "Monitor fungal diseases and apply preventive fungicide.",
        "issue_title_mr": "उच्च आर्द्रता",
        "advice_mr": "बुरशीजन्य रोगांवर लक्ष ठेवा आणि प्रतिबंधात्मक बुरशीनाशक वापरा."
    }
}

marathi_risk_levels = {
    "LOW": "कमी",
    "MEDIUM": "मध्यम",
    "HIGH": "उच्च"
}

marathi_phrases = {
    "overall_risk_level": "एकूण जोखीम पातळी:",
    "issue": "समस्या:",
    "risk_level": "जोखीम पातळी:",
    "recommendation": "शिफारस:",
    "enter_all_values": "कृपया सर्व हवामान मूल्ये प्रविष्ट करा.",
    "no_immediate_risk": "हवामानाची स्थिती सामान्य आहे. कोणतीही तात्काळ जोखीम आढळली नाही."
}



def generate_advisory(rainfall, temperature, humidity):

    if rainfall is None or temperature is None or humidity is None:
        return f" Please enter all climate values.\n\n {marathi_phrases['enter_all_values']}"

    risks = []

    for issue_key, data in climate_rules.items():
        if data["check"](rainfall, temperature, humidity):

            risks.append((issue_key, data["risk"], data["advice_en"], data["issue_title_mr"], data["advice_mr"]))

    if not risks:
        return f" Climate conditions are normal. No immediate risk detected.\n\n {marathi_phrases['no_immediate_risk']}"

    risk_priority = {"LOW": 1, "MEDIUM": 2, "HIGH": 3}
    overall_risk_en = max(risks, key=lambda x: risk_priority[x[1]])[1]
    overall_risk_mr = marathi_risk_levels[overall_risk_en]

    message_en = f"--- English Advisory ---\n"
    message_en += f" Overall Risk Level: {overall_risk_en}\n\n"

    message_mr = f"--- मराठी सल्ला ---\n"
    message_mr += f" {marathi_phrases['overall_risk_level']} {overall_risk_mr}\n\n"

    for issue_key, risk_en, advice_en, issue_title_mr, advice_mr in risks:
        risk_mr = marathi_risk_levels[risk_en]

        message_en += f" Issue: {issue_key.replace('_',' ').title()}\n"
        message_en += f"   Risk Level: {risk_en}\n"
        message_en += f"   Recommendation: {advice_en}\n\n"

        message_mr += f" {marathi_phrases['issue']} {issue_title_mr}\n"
        message_mr += f"   {marathi_phrases['risk_level']} {risk_mr}\n"
        message_mr += f"   {marathi_phrases['recommendation']} {advice_mr}\n\n"

    final_message = message_en + "\n" + message_mr
    return final_message


gr.Interface(
    fn=generate_advisory,
    inputs=[
        gr.Number(label="Rainfall (mm)"),
        gr.Number(label="Temperature (°C)"),
        gr.Number(label="Humidity (%)")
    ],
    outputs=gr.Textbox(label="Climate Risk Advisory (English & Marathi)", lines=15),
    title="KrushiAI – Climate Risk Advisory"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af21bd5dbb18cb0d38.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**KrushiAI Assist – Farmer Decision Support**

In [19]:
import gradio as gr

farmer_menu = {
    "1": {
        "title_en": "Crop damage due to heavy rain",
        "advice_en": [
            "Improve field drainage immediately",
            "Avoid fertilizer application",
            "Spray fungicide to prevent root rot",
            "Remove water stagnation within 24 hours"
        ],
        "title_mr": "अतिवृष्टीमुळे पिकांचे नुकसान",
        "advice_mr": [
            "शेतातील पाण्याचा निचरा त्वरित सुधारा",
            "खतांचा वापर टाळा",
            "मूळ कुज थांबवण्यासाठी बुरशीनाशक फवारणी करा",
            "24 तासांच्या आत साचलेले पाणी काढून टाका"
        ]
    },
    "2": {
        "title_en": "Heat stress in crops",
        "advice_en": [
            "Irrigate during early morning or evening",
            "Apply mulching to retain soil moisture",
            "Use shade nets if available",
            "Avoid chemical spraying during afternoon"
        ],
        "title_mr": "पिकांना उष्णतेचा ताण",
        "advice_mr": [
            "सकाळ किंवा संध्याकाळच्या वेळी सिंचन करा",
            "जमिनीतील ओलावा टिकवण्यासाठी मल्चिंगचा वापर करा",
            "उपलब्ध असल्यास शेड नेट वापरा",
            "दुपारच्या वेळी रासायनिक फवारणी टाळा"
        ]
    },
    "3": {
        "title_en": "Pest or disease attack",
        "advice_en": [
            "Identify pest before spraying",
            "Use neem-based pesticides first",
            "Avoid overuse of chemicals",
            "Consult local Krushi Seva Kendra if severe"
        ],
        "title_mr": "कीड किंवा रोग हल्ला",
        "advice_mr": [
            "फवारणी करण्यापूर्वी किडीची ओळख करा",
            "प्रथम कडुलिंबावर आधारित कीटकनाशके वापरा",
            "रसायनांचा अतिवापर टाळा",
            "तीव्र असल्यास स्थानिक कृषी सेवा केंद्राचा सल्ला घ्या"
        ]
    },
    "4": {
        "title_en": "Low yield problem",
        "advice_en": [
            "Check soil nutrient levels",
            "Use recommended crop variety",
            "Follow proper spacing",
            "Avoid excess fertilizer usage"
        ],
        "title_mr": "कमी उत्पादन समस्या",
        "advice_mr": [
            "मातीतील पोषक तत्वांची पातळी तपासा",
            "शिफारस केलेल्या पिकांची वाण वापरा",
            "योग्य अंतर राखा",
            "अतिरिक्त खतांचा वापर टाळा"
        ]
    }
}

def krushiai_assist(menu_choice):

    # Strip whitespace from the input to handle cases like " 1 "
    menu_choice = menu_choice.strip()

    if menu_choice not in farmer_menu:
        return "Invalid choice. Please select 1, 2, 3, or 4.\n\n अयोग्य निवड. कृपया 1, 2, 3, किंवा 4 निवडा."

    selected = farmer_menu[menu_choice]


    response_en = f"--- English Advisory ---\n"
    response_en += f" Issue Selected: {selected['title_en']}\n\n"
    response_en += " Recommended Actions:\n"
    for i, advice in enumerate(selected["advice_en"], start=1):
        response_en += f"{i}. {advice}\n"

    response_mr = f"--- मराठी सल्ला ---\n"
    response_mr += f" निवडलेली समस्या: {selected['title_mr']}\n\n"
    response_mr += "शिफारस केलेल्या कृती:\n"
    for i, advice in enumerate(selected["advice_mr"], start=1):
        response_mr += f"{i}. {advice}\n"

    return response_en + "\n" + response_mr

gr.Interface(
    fn=krushiai_assist,
    inputs=gr.Textbox(
        label="Select an option (1–4) / पर्याय निवडा (1-4)",
        placeholder="1 = Rain damage / पाऊस नुकसान | 2 = Heat stress / उष्णतेचा ताण | 3 = Pest / कीड | 4 = Low yield / कमी उत्पादन",
        lines=2
    ),
    outputs=gr.Textbox(label="KrushiAI Assist Output (English & Marathi)", lines=10),
    title="KrushiAI Assist – Farmer Decision Support"
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db5eebc6d10f0538d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
